In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [35]:
data = pd.read_csv(r'train.csv',encoding='latin1',on_bad_lines='skip',lineterminator='\n')

In [36]:
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [40]:
import re

def clean_text(text):
    text = text.lower()
    text = text.strip(' ') # Removing leading and trailing white spaces
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub('^[a-zA-Z]+',  " ",str(text))# Search for all non-letters,Replace all non-letters with spaces

    return text

In [41]:
data['comment_text'] = data['comment_text'].apply(lambda text : clean_text(text))

In [42]:
#Split the data
x = data['comment_text']
y = data.iloc[:,2:].values

In [43]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import TextVectorization

In [44]:
max_features = 100000

In [45]:
vectorizer = TextVectorization(max_tokens=max_features,
                               output_sequence_length=1800,
                               output_mode='int')

In [46]:
vectorizer.adapt(x.values)

In [47]:
vectorized_text = vectorizer(x.values)

In [48]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[   81,     2,   123, ...,     0,     0,     0],
       [   54,  2491,    14, ...,     0,     0,     0],
       [  437,    73,   130, ...,     0,     0,     0],
       ...,
       [10690,   390,    51, ...,     0,     0,     0],
       [   12,   546,    49, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [49]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [50]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

Bidrectional LSTM

In [51]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [52]:
model = Sequential()
# Create the embedding layer
model.add(Embedding(max_features+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer
model.add(Dense(6, activation='sigmoid'))

In [53]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam', metrics='Accuracy')

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          3200032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [55]:
history = model.fit(train, epochs=2, validation_data=val)

Epoch 1/2
6981/6981 [==============================] - 1988s 278ms/step - loss: 0.0642 - Accuracy: 0.9816 - val_loss: 0.0463 - val_Accuracy: 0.9939
Epoch 2/2
6981/6981 [==============================] - 1796s 257ms/step - loss: 0.0484 - Accuracy: 0.9898 - val_loss: 0.0446 - val_Accuracy: 0.9943


In [56]:
input_text=vectorizer('You freakin suck! i am going to hit you.')

In [57]:
res=model.predict(np.expand_dims(input_text, axis=0))

1/1 [==============================] - 6s 6s/step


In [58]:
res=model.predict(np.expand_dims(input_text, axis=0))

1/1 [==============================] - 0s 215ms/step


In [59]:
batch_x, batch_y = test.as_numpy_iterator().next()

In [60]:
(model.predict(batch_x)>0.5).astype(int)

1/1 [==============================] - 0s 288ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [33]:
res.shape

(1, 6)

In [61]:
oss, Acc = model.evaluate(test)

print('Test Accuracy: ', Acc)

997/997 [==============================] - 104s 103ms/step - loss: 0.0430 - Accuracy: 0.9935
Test Accuracy:  0.9935431480407715


In [62]:
from keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

for batch in test.as_numpy_iterator():
    # unpack the batch
    X_test, y_test = batch
    # make a prediction
    y_pred = model.predict(X_test)

    y_test = y_test.flatten()
    y_pred = y_pred.flatten()

    pre.update_state(y_test, y_pred)
    re.update_state(y_test, y_pred)
    acc.update_state(y_test, y_pred)

1/1 [==============================] - 0s 78ms/step


In [63]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8423680663108826, Recall:0.6756293177604675, Accuracy:0.49949848651885986


TEST AND GRADIO

In [64]:
import gradio as gr

In [65]:
model.save('toxicity.h5')

In [66]:
model2=tf.keras.models.load_model('toxicity.h5')

In [67]:
input_str=vectorizer('Hey I freaking hate you!')

In [68]:
res=model.predict(np.expand_dims(input_str, 0))

1/1 [==============================] - 1s 557ms/step


In [69]:
res

array([[0.93072784, 0.08280525, 0.5726236 , 0.03885945, 0.61217254,
        0.12116433]], dtype=float32)

In [73]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(data.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text

In [74]:
interface = gr.Interface(fn=score_comment,
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [75]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://264770be23d990b7a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 94ms/step
